In [8]:
# Ignore some warning messages
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import numpy as np
import jieba
import ujson
import pandas as pd
from jieba import lcut
from tqdm import tqdm
from pprint import pprint
from collections import Counter
from tools import flatten

from tools import dereplication
from tools import punctuation_line
from gensim.models import KeyedVectors
import matplotlib.pylab as plt


# Set the log print content
import logging

logging.basicConfig(format='%(asctime)s:%(levelname)s: %(message)s', level=logging.INFO)
jieba.setLogLevel(logging.INFO)

In [9]:
# Load data file into memory
def loaddata(filename, type='train'):
    with open(filename, 'r', encoding='utf8') as file:
        train_data = ujson.loads(file.read())

    questions = []
    answers = []
    documents = []
    for _, value in train_data.items():
        question = value['question']
        for _, evidence in value['evidences'].items():
            questions.append(question)
            answer = ''.join(evidence['answer'])
            answers.append(answer)
            document = evidence['evidence']
            documents.append(document)

    train_simple = map(lambda x, y, z: [x, y, z], questions, answers, documents)

    train_df = pd.DataFrame([item for item in train_simple], columns=['question', 'answer', 'document'])
    print(train_df.describe())
    train_df.to_csv('train' + '.csv', index=False, header=True)

    return [questions, answers, documents, train_simple]

In [10]:
class DataPath:
    def __init__(self):
        self.train_data_path = 'data/me_train.json'
        self.test_data_path = 'data/me_test.ann.json'
        self.validation_data_path = 'data/me_validation.ann.json'

In [11]:
data_path = DataPath()
train_simples = loaddata(data_path.train_data_path)

             question     answer                 document
count          448444     448444                   448444
unique          36174      14910                   408410
top     南浦大桥于哪一年建成通车？  no_answer  1884年7月6日，自由女神像正式赠送给美国。
freq               29     307547                       25